In [ ]:
import os
os.chdir("..")

In [ ]:
from src.data.data_process import DataReg
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
from linearmodels.panel import RandomEffects, PanelOLS
import polars as pl

dr = DataReg()

In [ ]:
df_qcew = dr.regular_data()

df_qcew.group_by(["year", "qtr", "zipcode", "ein"]).agg(
           total_employment=pl.col("total_employment").sum(),
            total_wages=pl.col("total_wages").sum()
        )
tmp = pl.from_pandas(dr.make_spatial_dataset().drop("geometry", axis=1))
tmp

In [ ]:
master = df_qcew.join(tmp,on=["year","qtr","zipcode"], how="inner", validate="m:1")
master

In [ ]:
data = master.to_pandas().copy()
data["date2"] = data["year"] * 10 + data["qtr"]
data['date'] = data['date2'].astype('category')
data['zipcode'] = data['zipcode'].astype('category')
data['ein'] = data['ein'].astype('category')

In [ ]:
model = bmb.Model(
    "total_employment ~ 0 + k_index + own_children6 + own_children17  + commute_car + food_stamp + with_social_security",
    data, dropna=True,
)
results = model.fit(sample_kwargs={"nuts_sampler" :"blackjax"})

In [ ]:
# Plot posteriors
az.plot_trace(
    results,
    compact=True,
)

In [ ]:
import jax

In [ ]:
print(jax.devices())